In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
import numpy as np
from cp2k_file_utils import read_energy
from pao_file_utils import parse_pao_file, write_pao_file
from pao_tfn_trainer import loss_function, ortho

In [4]:
frame_dir = "2H2O_MD/frame_0000"
kinds, atom2kind, coords, xblocks = parse_pao_file(frame_dir + "/2H2O_pao44-1_0.pao")

for i in range(11):
    eps = i * 0.01
    losses = []
    perturbed_xblocks = []
    for xblock in xblocks:
        perturbed_xblock = torch.as_tensor(xblock + eps*np.random.rand(*xblock.shape))
        perturbed_xblock = ortho(perturbed_xblock)
        perturbed_xblocks.append(perturbed_xblock)
        losses.append(loss_function(perturbed_xblock, torch.as_tensor(xblock)))
    loss = np.mean(losses)

    write_pao_file(frame_dir + "/inferred.pao", kinds, atom2kind, coords, perturbed_xblocks)
    ! cd $frame_dir; /opt/cp2k/exe/local/cp2k.ssmp -i 2H2O_pao44_inferred.inp > 2H2O_pao44_inferred.out
    energy = read_energy(frame_dir+"/2H2O_pao44_inferred.out")
    
    print("energy: {}  loss: {:g}  eps: {:g}".format(energy, loss, eps))

energy: -34.44993613896875  loss: 8.68858e-33  eps: 0
energy: -34.44524978438085  loss: 1.15119e-05  eps: 0.01
energy: -34.436378674603276  loss: 3.04936e-05  eps: 0.02
energy: -34.41486864907458  loss: 8.80838e-05  eps: 0.03
energy: -34.39795217912157  loss: 0.000123866  eps: 0.04
energy: -34.3498496047107  loss: 0.000245133  eps: 0.05
energy: -34.285281439836254  loss: 0.000389081  eps: 0.06
energy: -34.26588777186778  loss: 0.000463375  eps: 0.07
energy: -34.22954852824688  loss: 0.000465812  eps: 0.08
energy: -34.13062862819209  loss: 0.000791106  eps: 0.09
energy: -34.06197967462564  loss: 0.00110367  eps: 0.1
